In [1]:
from loader import MoleculeDataset, SeqDataset,SeqMolDataset,SmileDataset#########################
import torch
import torch
#import args
from torchvision.models import resnet18

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse

from loader import MoleculeDataset#################
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
import numpy as np

from model import GNN, GNN_graphpred,GNN_graphpred_1
from sklearn.metrics import roc_auc_score

from splitters import scaffold_split,scaffold_split_1
import pandas as pd
import os
import shutil
from tensorboardX import SummaryWriter
import esm

from SeqMolModel import InteractionModel,InteractionModel_1,SequenceModel,InteractionModel_4

In [16]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch implementation of pre-training of graph neural networks')
parser.add_argument('--device', type=int, default=0,
                        help='which gpu to use if any (default: 0)')#0000
parser.add_argument('--batch_size', type=int, default=1,
                        help='input batch size for training (default: 32)')
parser.add_argument('--epochs', type=int, default=100,
                        help='number of epochs to train (default: 100)')
parser.add_argument('--lr', type=float, default=0.001,
                        help='learning rate (default: 0.1)')
parser.add_argument('--lr_scale', type=float, default=1,
                        help='relative learning rate for the feature extraction layer (default: 1)')
parser.add_argument('--decay', type=float, default=0,
                        help='weight decay (default: 0)')
parser.add_argument('--num_layer', type=int, default=5,
                        help='number of GNN message passing layers (default: 5).')
parser.add_argument('--emb_dim', type=int, default=300,
                        help='embedding dimensions (default: 300)')
parser.add_argument('--dropout_ratio', type=float, default=0.5,
                        help='dropout ratio (default: 0.5)')
parser.add_argument('--graph_pooling', type=str, default="mean",
                        help='graph level pooling (sum, mean, max, set2set, attention)')
parser.add_argument('--JK', type=str, default="last",
                        help='how the node features across layers are combined. last, sum, max or concat')
parser.add_argument('--gnn_type', type=str, default="gin")
parser.add_argument('--dataset', type=str, default = 'affinity', help='root directory of dataset. For now, only classification.')
#parser.add_argument('--input_model_file', type=str, default = 'None', help='filename to read the model (if there is any)')
parser.add_argument('--input_model_file', type=str, default = 'Mole-BERT', help='filename to read the model (if there is any)')
parser.add_argument('--filename', type=str, default = '', help='output filename')
parser.add_argument('--seed', type=int, default=42, help = "Seed for splitting the dataset.")
parser.add_argument('--runseed', type=int, default=0, help = "Seed for minibatch selection, random initialization.")
parser.add_argument('--split', type = str, default="scaffold", help = "random or scaffold or random_scaffold")
parser.add_argument('--eval_train', type=int, default = 1, help='evaluating training or not')
parser.add_argument('--num_workers', type=int, default = 4, help='number of workers for dataset loading')

parser.add_argument('--n_head', type=int, default = 12, help='number of workers for dataset loading')

parser.add_argument('--n_layer', type=int, default = 12, help='number of workers for dataset loading')
parser.add_argument('--d_dropout', type=float, default = 0.1, help='number of workers for dataset loading')
parser.add_argument('--n_embd', type=int, default = 768, help='number of workers for dataset loading')
parser.add_argument('--dropout', type=float, default = 0.1, help='number of workers for dataset loading')
parser.add_argument('--lr_start', type=float, default =  3e-5, help='number of workers for dataset loading')
parser.add_argument('--max_epochs', type=int, default = 500, help='number of workers for dataset loading')
parser.add_argument('--num_feats', type=int, default = 32, help='number of workers for dataset loading')
parser.add_argument('--checkpoint_every', type=int, default = 100, help='number of workers for dataset loading')
parser.add_argument('--seed_path', type=str, default =  'data/checkpoints/N-Step-Checkpoint_3_30000.ckpt', help='number of workers for dataset loading')
parser.add_argument('--dims', type=list, default = [ 768, 768, 768, 1], help='number of workers for dataset loading')

args = parser.parse_args(args=[])###############33


In [3]:
#load pretained model of Mole-Bert


device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.runseed)

num_tasks=1
# Load ESM-2 model
protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
protein_model.to(device)
#freezing parameters
for i,p in enumerate(protein_model.parameters()):
    p.requires_grad = False
#print(protein_alphabet)
#alphabet = esm.Alphabet.from_architecture(model_data["args"].arch)
#batch_converter = alphabet.get_batch_converter()
protein_model.eval()  # disables dropout for deterministic results

#self.molecular.model,self.molecular.node_representation,self.molecular.features = GNN_graphpred_1(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
molecular_model = GNN_graphpred_1(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
###################################
if not args.input_model_file == "None":###############
    print('Not from scratch')
    molecular_model.from_pretrained('model_gin/{}.pth'.format(args.input_model_file))
    print('rese:model_gin')
molecular_model.to(device)
for i,p in enumerate(molecular_model.parameters()):
    p.requires_grad = False#freezing parameters
#freezing parameters
for i,p in enumerate(protein_model.parameters()):
    p.requires_grad = False#freezing parameters

'''
model_param_group = []
model_param_group.append({"params": molecular_model.gnn.parameters()})
if args.graph_pooling == "attention":
    model_param_group.append({"params": molecular_model.pool.parameters(), "lr":args.lr*args.lr_scale})
'''
'''
————————————————
版权声明：本文为CSDN博主「Dreamcatcher风」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/Wind_2028/article/details/120541017   
'''


'''
model_param_group.append({"params": molecular_model.graph_pred_linear.parameters(), "lr":args.lr*args.lr_scale})
optimizer = optim.Adam(model_param_group, lr=0.01, weight_decay=args.decay)#############

'''




Not from scratch
rese:model_gin


'\nmodel_param_group.append({"params": molecular_model.graph_pred_linear.parameters(), "lr":args.lr*args.lr_scale})\noptimizer = optim.Adam(model_param_group, lr=0.01, weight_decay=args.decay)#############\n\n'

In [4]:
import time
import torch
from torch import nn
import argparse
import torch.nn.functional as F
import os
import numpy as np
import random
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_warn, rank_zero_only, seed
#from finetune.tokenizer.tokenizer import MolTranBertTokenizer
from fast_transformers.masking import LengthMask as LM
#from rotate_attention.rotate_builder import RotateEncoderBuilder as rotate_builder
from fast_transformers.feature_maps import GeneralizedRandomFeatures
from functools import partial
from apex import optimizers
import subprocess
from argparse import ArgumentParser, Namespace
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from torch.utils.data import DataLoader
from sklearn.metrics import r2_score
#from utils import normalize_smiles
import sys
sys.path.append('finetune/')
from utilss import normalize_smiles
from tokenizer.tokenizer import MolTranBertTokenizer
from rotate_attention.rotate_builder import RotateEncoderBuilder as rotate_builder
#from utils import normalize_smiles
# create a function (this my favorite choice)
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))


class LightningModule(pl.LightningModule):

    def __init__(self, config, tokenizer):
        super(LightningModule, self).__init__()

        self.config = config
        #self.hparams = config
        #self.mode = config.mode
        self.save_hyperparameters(config)
        self.tokenizer=tokenizer
        '''
        self.min_loss = {
            self.hparams.measure_name + "min_valid_loss": torch.finfo(torch.float32).max,
            self.hparams.measure_name + "min_epoch": 0,
        }
        '''
        # Word embeddings layer
        n_vocab, d_emb = len(tokenizer.vocab), config.n_embd
        # input embedding stem
        
        builder = rotate_builder.from_kwargs(
            n_layers=config.n_layer,
            n_heads=config.n_head,
            query_dimensions=config.n_embd//config.n_head,
            value_dimensions=config.n_embd//config.n_head,
            feed_forward_dimensions=config.n_embd,
            attention_type='linear',
            feature_map=partial(GeneralizedRandomFeatures, n_dims=config.num_feats),
            activation='gelu',
            )
        self.pos_emb = None
        self.tok_emb = nn.Embedding(n_vocab, config.n_embd)
        #print('self.tok_emb:',self.tok_emb)
        self.drop = nn.Dropout(config.d_dropout)
        
        ## transformer
        self.blocks = builder.get()
        #self.lang_model = self.lm_layer(config.n_embd, n_vocab)
        #self.train_config = config
        #if we are starting from scratch set seeds
        #########################################
        # protein_emb_dim, smiles_embed_dim, dims=dims, dropout=0.2):
        #########################################
        '''
        self.fcs = []  
        self.loss = torch.nn.L1Loss()
        self.net = self.Net(
            config.n_embd, dims=config.dims, dropout=config.dropout,
        )
        '''


    class Net(nn.Module):
        dims = [150, 50, 50, 2]


        def __init__(self, smiles_embed_dim, dims=dims, dropout=0.2):
            super().__init__()
            self.desc_skip_connection = True 
            self.fcs = []  # nn.ModuleList()
            #print('dropout is {}'.format(dropout))

            self.fc1 = nn.Linear(smiles_embed_dim, smiles_embed_dim)
            self.dropout1 = nn.Dropout(dropout)
            self.relu1 = nn.GELU()
            self.fc2 = nn.Linear(smiles_embed_dim, smiles_embed_dim)
            self.dropout2 = nn.Dropout(dropout)
            self.relu2 = nn.GELU()
            self.final = nn.Linear(smiles_embed_dim, 1)

        def forward(self, smiles_emb):
            x_out = self.fc1(smiles_emb)
            x_out = self.dropout1(x_out)
            x_out = self.relu1(x_out)

            if self.desc_skip_connection is True:
                x_out = x_out + smiles_emb

            z = self.fc2(x_out)
            z = self.dropout2(z)
            z = self.relu2(z)
            if self.desc_skip_connection is True:
                z = self.final(z + x_out)
            else:
                z = self.final(z)

            return z

    class lm_layer(nn.Module):
        def __init__(self, n_embd, n_vocab):
            super().__init__()
            self.embed = nn.Linear(n_embd, n_embd)
            self.ln_f = nn.LayerNorm(n_embd)
            self.head = nn.Linear(n_embd, n_vocab, bias=False)
        def forward(self, tensor):
            tensor = self.embed(tensor)
            tensor = F.gelu(tensor)
            tensor = self.ln_f(tensor)
            tensor = self.head(tensor)
            return tensor

    def get_loss(self, smiles_emb, measures):

        z_pred = self.net.forward(smiles_emb).squeeze()
        measures = measures.float()

        return self.loss(z_pred, measures), z_pred, measures
    

In [5]:
margs = args
tokenizer = MolTranBertTokenizer('finetune/bert_vocab.txt')
seed.seed_everything(margs.seed)
if margs.seed_path == '':
    #print("# training from scratch")
    smile_model = LightningModule(margs, tokenizer)
else:
    #print("# loaded pre-trained model from {args.seed_path}")
    smile_model = LightningModule(margs, tokenizer).load_from_checkpoint(margs.seed_path, strict=False, config=margs, tokenizer=tokenizer, vocab=len(tokenizer.vocab))#########################33

#print('model:',smile_model)
#freezing parameters
for i,p in enumerate(smile_model.parameters()):
    p.requires_grad = False

Global seed set to 42


Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['lang_model.embed.weight', 'lang_model.embed.bias', 'lang_model.ln_f.weight', 'lang_model.ln_f.bias', 'lang_model.head.weight']
  rank_zero_warn(


In [70]:
#m=LightningModule(margs, tokenizer)
#print(m)
#smile_model=smile_model_all.blocks
#print(smile_model)

#tokenizer=MolTranBertTokenizer('bert_vocab.txt')

In [6]:
class PMLP():
    def __init__(self):
        self.fc1=nn.Linear(1,100)
        self.relu1=nn.ReLU()
        self.fc2=nn.Linear(100,1)
        self.sig=nn.Sigmoid()
        self.dropout=nn.Dropout(0.5)
    def forward(x):
        out=self.dropout(self.relu1(self.fc1(x)))
        out=self.sig(self.fc2(out))
        return out
pmodel=PMLP()

In [6]:
#num_tasks=1
model= InteractionModel_4(protein_model=protein_model,molecular_model=molecular_model,smile_model=smile_model,protein_embd_dim=1280,num_tasks=1,device=device,mol_embd_dim=300,smile_embd_dim=768) 
model.to(device)
#print(model)#nice

InteractionModel_4(
  (protein_model): ESM2(
    (embed_tokens): Embedding(33, 1280, padding_idx=1)
    (layers): ModuleList(
      (0): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (rot_emb): RotaryEmbedding()
        )
        (self_attn_layer_norm): ESM1bLayerNorm(torch.Size([1280]), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (final_layer_norm): ESM1bLayerNorm(torch.Size([1280]), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_fea

In [7]:
###################when doing geometric.data process, if there is some changes, please delete the directory process and let it generate again

gnn_dataset = MoleculeDataset("./dataset/" + args.dataset, dataset=args.dataset)###########################转换成了分子图的格式
#print('args.dataset:',args.dataset)
#print(gnn_dataset)
#for i,gnn_data in enumerate(gnn_dataset):
    #print(gnn_data)

In [9]:
'''
#test gnn_dataset whether is right
for i ,data in enumerate(gnn_dataset):
    print(data.x)
'''

'\n#test gnn_dataset whether is right\nfor i ,data in enumerate(gnn_dataset):\n    print(data.x)\n'

In [8]:
seq_dataset=SeqDataset('dataset/affinity/processed/sequence.csv')
#smiles_dataset=SmileDataset('dataset/affinity/processed/smiles.csv')

#seq_dataset[2]

In [23]:
'''
class SmileDataset(torch.utils.data.Dataset):
    def __init__(self,smilePath):
        if type(smilePath)==str:
            self.df_file=pd.read_csv(smilePath,sep=',',header=None)
            self.smile_list=self.df_file.iloc[:,0]
        else:
            self.df=pd.DataFrame(smilePath)
            self.smile_list=self.df.iloc[:,0].tolist()
        #self.sequence_tensor=torch.tensor(self.sequence_list)
        self.original_smiles = []
        self.original_canonical_map = {
            smi: normalize_smiles(smi, canonical=True, isomeric=False) for smi in self.smile_list
        }
        #print('self.original_canonical_map:',self.original_canonical_map)
        #self.tokenizer = MolTranBertTokenizer('finetune/bert_vocab.txt')
        for i in range(len(self.smile_list)):
            smi = self.smile_list[i]
            if smi in self.original_canonical_map.keys():
                self.original_smiles.append(smi)
        #print('self.original_smiles:',self.original_smiles)
        print(f"Embeddings not found for {len(self.smile_list) - len(self.original_smiles)} molecules")
        
        
    def __getitem__(self, idx):
        #smile=self.smile_list[idx]
        print('idx:',idx)
        original_smiles = self.original_smiles[idx]
        
        canonical_smiles = self.original_canonical_map[original_smiles]
        return canonical_smiles
        #return smile
    
    def __len__(self):
        return len(self.original_smiles)
'''  

In [37]:
'''
class SmileDataset(torch.utils.data.Dataset):
    def __init__(self,smilePath):
        if type(smilePath)==str:
            self.df_file=pd.read_csv(smilePath,sep=',',header=None)
            self.smile_list=self.df_file.iloc[:,0]
        else:
            self.df=pd.DataFrame(smilePath)
            self.smile_list=self.df.iloc[:,0].tolist()
        #self.sequence_tensor=torch.tensor(self.sequence_list)
        #self.original_smiles = []
        #self.original_canonical_map = {
        #    smi: normalize_smiles(smi, canonical=True, isomeric=False) for smi in self.smile_list
        #}
        #print('self.original_canonical_map:',self.original_canonical_map)
        #self.tokenizer = MolTranBertTokenizer('finetune/bert_vocab.txt')
        #for i in range(len(self.smile_list)):
        #   smi = self.smile_list[i]
        #    if smi in self.original_canonical_map.keys():
        #        self.original_smiles.append(smi)
        #print('self.original_smiles:',self.original_smiles)
        #print(f"Embeddings not found for {len(self.smile_list) - len(self.original_smiles)} molecules")
        
        
    def __getitem__(self, idx):
        smile=self.smile_list[idx]
        print('idx:',idx)
        #original_smiles = self.original_smiles[idx]
        
        #canonical_smiles = self.original_canonical_map[original_smiles]
        #return canonical_smiles
        return smile
        #return smile
    
    def __len__(self):
        return len(self.smile_list)
'''

'\nclass SmileDataset(torch.utils.data.Dataset):\n    def __init__(self,smilePath):\n        if type(smilePath)==str:\n            self.df_file=pd.read_csv(smilePath,sep=\',\',header=None)\n            self.smile_list=self.df_file.iloc[:,0]\n        else:\n            self.df=pd.DataFrame(smilePath)\n            self.smile_list=self.df.iloc[:,0].tolist()\n        #self.sequence_tensor=torch.tensor(self.sequence_list)\n        #self.original_smiles = []\n        #self.original_canonical_map = {\n        #    smi: normalize_smiles(smi, canonical=True, isomeric=False) for smi in self.smile_list\n        #}\n        #print(\'self.original_canonical_map:\',self.original_canonical_map)\n        #self.tokenizer = MolTranBertTokenizer(\'finetune/bert_vocab.txt\')\n        #for i in range(len(self.smile_list)):\n        #   smi = self.smile_list[i]\n        #    if smi in self.original_canonical_map.keys():\n        #        self.original_smiles.append(smi)\n        #print(\'self.original_smile

In [17]:
'''
class SmileDataset(torch.utils.data.Dataset):
    def __init__(self,smilePath):
        if type(smilePath)==str:
            self.df_file=pd.read_csv(smilePath,sep=',',header=None)
            self.smile_list=self.df_file.iloc[:,0]
        else:
            self.df=pd.DataFrame(smilePath)
            self.smile_list=self.df.iloc[:,0].tolist()
        print('self.smile_list:',type(self.smile_list))
        #self.sequence_tensor=torch.tensor(self.sequence_list)
        self.original_smiles = []
        self.original_canonical_map = {
            smi: normalize_smiles(smi, canonical=True, isomeric=False) for smi in self.smile_list
        }
        #print('self.original_canonical_map:',self.original_canonical_map)
        #self.tokenizer = MolTranBertTokenizer('finetune/bert_vocab.txt')
        for i in range(len(self.smile_list)):
            smi = self.smile_list[i]
            if smi in self.original_canonical_map.keys():
                self.original_smiles.append(smi)
        self.original_smiles_1=pd.Series(self.original_smiles)
        self.original_canonical_map_1=pd.Series(self.original_canonical_map)
        #print('self.original_smiles:',self.original_smiles)
        print(f"Embeddings not found for {len(self.smile_list) - len(self.original_smiles)} molecules")
        
        
    def __getitem__(self, idx):
        #smile=self.smile_list[idx]
        print('idx:',idx)
        original_smiles = self.original_smiles_1[idx]
        
        #canonical_smiles = self.original_canonical_map[original_smiles]
        #return canonical_smiles
        #return original_smiles
    
    def __len__(self):
        return len(self.original_smiles)
'''

'\nclass SmileDataset(torch.utils.data.Dataset):\n    def __init__(self,smilePath):\n        if type(smilePath)==str:\n            self.df_file=pd.read_csv(smilePath,sep=\',\',header=None)\n            self.smile_list=self.df_file.iloc[:,0]\n        else:\n            self.df=pd.DataFrame(smilePath)\n            self.smile_list=self.df.iloc[:,0].tolist()\n        print(\'self.smile_list:\',type(self.smile_list))\n        #self.sequence_tensor=torch.tensor(self.sequence_list)\n        self.original_smiles = []\n        self.original_canonical_map = {\n            smi: normalize_smiles(smi, canonical=True, isomeric=False) for smi in self.smile_list\n        }\n        #print(\'self.original_canonical_map:\',self.original_canonical_map)\n        #self.tokenizer = MolTranBertTokenizer(\'finetune/bert_vocab.txt\')\n        for i in range(len(self.smile_list)):\n            smi = self.smile_list[i]\n            if smi in self.original_canonical_map.keys():\n                self.original_smil

In [9]:
 def collate(batch):
        #print('collate_batch:',batch)
        #print('collate_batch_0:',batch[0])
        tokenizer = MolTranBertTokenizer('finetune/bert_vocab.txt')
        
        tokens = tokenizer.batch_encode_plus([ smile for smile in batch], padding=True, add_special_tokens=True)
        #print('tokens[1]_mask:',tokens[1])
        #print('colate###########################')
        #for i,m in enumerate(tokens):
            
        #print('collate_tokens_input_ids:',tokens['input_ids'])
        #print('colate_tokens_attention_mask:',tokens['attention_mask'])
        return (torch.tensor(tokens['input_ids']), torch.tensor(tokens['attention_mask']))

In [10]:
smiles_dataset=SmileDataset('dataset/affinity/processed/smiles.csv')


In [16]:
#smile_DataLoader=DataLoader(smiles_dataset, batch_size=1, shuffle=False, num_workers=0, collate_fn=collate, pin_memory=False, drop_last=False)

In [26]:
for i in range(1):
    for j,( a,b) in enumerate(smile_DataLoader):
        print(a)
        print(b)
        break

idx: 0
tensor([[ 0, 10,  1]])
tensor([[1, 1, 1]])


In [12]:
######33test seq_data,for there is error :data fomat: [(name,sequence),(name,sequence),()]
'''
seq_dataloader=DataLoader(seq_dataset,batch_size=args.batch_size,shuffle=True,num_workers=args.num_workers)

#protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()

#print(protein_alphabet)
#alphabet = esm.Alphabet.from_architecture(model_data["args"].arch)
#batch_converter = alphabet.get_batch_converter()
#protein_model.eval()  # disables dropout for deterministic results

#batch_converter=protein_alphabet.get_batch_converter()
protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter=protein_alphabet.get_batch_converter()

for i ,seq in enumerate(seq_dataloader):
    #seq_list=seq_model(seq)
    protein_sequence_representations = []
    
    batch_labels, batch_strs, batch_tokens = batch_converter(seq)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    with torch.no_grad():
        results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)
        
    protein_token_representations = results["representations"][33]
    print('protein_token_representations:',protein_token_representations)
    for i, tokens_len in enumerate(batch_lens):
        protein_sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
    
'''

'\nseq_dataloader=DataLoader(seq_dataset,batch_size=args.batch_size,shuffle=True,num_workers=args.num_workers)\n\n#protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()\n\n#print(protein_alphabet)\n#alphabet = esm.Alphabet.from_architecture(model_data["args"].arch)\n#batch_converter = alphabet.get_batch_converter()\n#protein_model.eval()  # disables dropout for deterministic results\n\n#batch_converter=protein_alphabet.get_batch_converter()\nprotein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()\nbatch_converter=protein_alphabet.get_batch_converter()\n\nfor i ,seq in enumerate(seq_dataloader):\n    #seq_list=seq_model(seq)\n    protein_sequence_representations = []\n    \n    batch_labels, batch_strs, batch_tokens = batch_converter(seq)\n    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)\n    with torch.no_grad():\n        results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)\n        \n    protein_token_representatio

In [11]:
#####test with chartGPT  DataSet extends two parents' classes

import torch
from torch.utils.data import Dataset
from torch_geometric.data import InMemoryDataset

class MultiDatasetMixin:
    def __init__(self, dataset1, dataset2,dataset3):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.dataset3=dataset3

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2),len(self.dataset3))

    def __getitem__(self, idx):
        
        return self.dataset1[idx], self.dataset2[idx],self.dataset3[idx]

class CustomMultiDataset(MultiDatasetMixin, Dataset):###############3extends two classes
    def __init__(self, dataset1, dataset2,dataset3):
        MultiDatasetMixin.__init__(self, dataset1, dataset2,dataset3)


'''
# Example usage
if __name__ == "__main__":

    
    class DummyDataset1(Dataset):
        def __len__(self):
            return 100

        def __getitem__(self, idx):
            return torch.rand(3)

    class DummyDataset2(InMemoryDataset):
        def __init__(self):
            super(DummyDataset2, self).__init__("path/to/processed/dataset")
            self.data = None  # Replace with actual data loading logic

    dataset1 = DummyDataset1()
    dataset2 = DummyDataset2()
    
    
    dataset1=seq_dataset
    dataset2=gnn_dataset
    custom_dataset = CustomMultiDataset(dataset1, dataset2)

    print(len(custom_dataset))
    print(custom_dataset[0])
'''

'\n# Example usage\nif __name__ == "__main__":\n\n    \n    class DummyDataset1(Dataset):\n        def __len__(self):\n            return 100\n\n        def __getitem__(self, idx):\n            return torch.rand(3)\n\n    class DummyDataset2(InMemoryDataset):\n        def __init__(self):\n            super(DummyDataset2, self).__init__("path/to/processed/dataset")\n            self.data = None  # Replace with actual data loading logic\n\n    dataset1 = DummyDataset1()\n    dataset2 = DummyDataset2()\n    \n    \n    dataset1=seq_dataset\n    dataset2=gnn_dataset\n    custom_dataset = CustomMultiDataset(dataset1, dataset2)\n\n    print(len(custom_dataset))\n    print(custom_dataset[0])\n'

In [12]:
#chartGPT太厉害了，杀了我吧
from torch.utils.data import DataLoader
from torch_geometric.data import DataLoader as GeometricDataLoader

class MultiDataLoader:
    def __init__(self, dataloader1, dataloader2,dataloader3):
        self.dataloader1 = dataloader1
        self.dataloader2 = dataloader2
        self.dataloader3=dataloader3

    def __iter__(self):
        for data1, data2,data3 in zip(self.dataloader1, self.dataloader2, self.dataloader3):
            yield data1, data2, data3

    def __len__(self):
        return min(len(self.dataloader1), len(self.dataloader2), len(self.dataloader3))

    def set_shuffle(self, shuffle):
        self.dataloader1.shuffle = shuffle
        self.dataloader2.shuffle = shuffle
        self.dataloader3.shuffle=shuffle
'''
# Example usage
if __name__ == "__main__":
    # Create DataLoader instances (replace with your actual datasets and configurations)
    dataloader1 = DataLoader(seq_dataset, batch_size=32, shuffle=False)######False
    dataloader2 = GeometricDataLoader(gnn_dataset, batch_size=32, shuffle=False)

    multi_loader = MultiDataLoader(dataloader1, dataloader2)

    # Set the shuffle parameter simultaneously for both dataloaders
    multi_loader.set_shuffle(True)

    for batch_data1, batch_data2 in multi_loader:
        # Process your batch data here
        print("Data from DataLoader 1:", batch_data1)
        print("Data from DataLoader 2:", batch_data2)
'''

'\n# Example usage\nif __name__ == "__main__":\n    # Create DataLoader instances (replace with your actual datasets and configurations)\n    dataloader1 = DataLoader(seq_dataset, batch_size=32, shuffle=False)######False\n    dataloader2 = GeometricDataLoader(gnn_dataset, batch_size=32, shuffle=False)\n\n    multi_loader = MultiDataLoader(dataloader1, dataloader2)\n\n    # Set the shuffle parameter simultaneously for both dataloaders\n    multi_loader.set_shuffle(True)\n\n    for batch_data1, batch_data2 in multi_loader:\n        # Process your batch data here\n        print("Data from DataLoader 1:", batch_data1)\n        print("Data from DataLoader 2:", batch_data2)\n'

In [13]:
###########split train dataset validate dataset test dataset
seq_gnn_dataset=MultiDatasetMixin(seq_dataset,gnn_dataset,smiles_dataset)

#print('seq_dataset:',seq_dataset)

#seq_dataloader=DataLoader(seq_dataset,batch_size=args.batch_size,shuffle=True,num_workers=args.num_workers)
'''
for i ,seq in enumerate(seq_dataloader):
    print(seq)
'''
if args.split == "scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        
        #print('smiles_list:',smiles_list)
        mol_train_dataset, mol_valid_dataset, mol_test_dataset ,seq_train_dataset,seq_valid_dataset,seq_test_dataset,smile_train_dataset,smile_valid_dataset,smile_test_dataset= scaffold_split_1(seq_gnn_dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1)##########dataset
        print("scaffold")
elif args.split == "random":
        train_dataset, valid_dataset, test_dataset = random_split(dataset, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        #print("random")
elif args.split == "random_scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        train_dataset, valid_dataset, test_dataset = random_scaffold_split(dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        #print("random scaffold")
else:
        raise ValueError("Invalid split option.")

#print('++++++++++', mol_train_dataset[0])


#seq_mol_train_dataset=MultiDatasetMixini(seq_train_dataset,mol_train_dataset)
#seq_mol_valid_dataset=SeqMolDataset(seq_valid_dataset,mol_valid_dataset)
#seq_mol_test_dataset=SeqMolDataset(seq_train_dataset,mol_test_dataset)
seq_train_dataloader1 = DataLoader(seq_train_dataset, batch_size=args.batch_size, shuffle=False,num_workers=args.num_workers)######False
mol_train_dataloader2 = GeometricDataLoader(mol_train_dataset, batch_size=args.batch_size, shuffle=False,num_workers=args.num_workers)
smile_train_dataloader3=DataLoader(smile_train_dataset,batch_size=args.batch_size,collate_fn=collate, shuffle=False,num_workers=args.num_workers)
seq_mol_smile_train_multi_loader = MultiDataLoader(seq_train_dataloader1, mol_train_dataloader2,smile_train_dataloader3)
# Set the shuffle parameter simultaneously for both dataloaders
seq_mol_smile_train_multi_loader.set_shuffle(True)
'''
print('seq_train_dataset#########:',seq_train_dataset)

for i,seq in enumerate(seq_train_dataloader1):
    print(seq)
'''
seq_valid_dataloader1 = DataLoader(seq_valid_dataset, batch_size=args.batch_size,shuffle=False,num_workers=args.num_workers)######False
mol_valid_dataloader2 = GeometricDataLoader(mol_valid_dataset, batch_size=args.batch_size, shuffle=False,num_workers=args.num_workers)
smile_valid_dataloader3=DataLoader(smile_valid_dataset, batch_size=args.batch_size,collate_fn=collate,shuffle=False,num_workers=args.num_workers)
for i,m in enumerate(smile_train_dataloader3):
    #print('m@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@:',m)
    break
seq_mol_smile_valid_multi_loader = MultiDataLoader(seq_valid_dataloader1, mol_valid_dataloader2,smile_valid_dataloader3)
# Set the shuffle parameter simultaneously for both dataloaders
seq_mol_smile_valid_multi_loader.set_shuffle(True)

seq_test_dataloader1 = DataLoader(seq_test_dataset, batch_size=args.batch_size, shuffle=False,num_workers=args.num_workers)######False
mol_test_dataloader2 = GeometricDataLoader(mol_test_dataset, batch_size=args.batch_size, shuffle=False,num_workers=args.num_workers)
smile_test_dataloader3=DataLoader(smile_test_dataset,batch_size=args.batch_size,collate_fn=collate, shuffle=False,num_workers=args.num_workers)

seq_mol_smile_test_multi_loader = MultiDataLoader(seq_test_dataloader1, mol_valid_dataloader2,smile_test_dataloader3)
# Set the shuffle parameter simultaneously for both dataloaders
seq_mol_smile_test_multi_loader.set_shuffle(True)


scaffold


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


"\nprint('mol_dataloader:')\nfor mol in mol_train_dataloader2:\n    print(mol)\nfor seq in seq_train_dataloader1:\n    print(seq)\n "

In [25]:
###########test SeqDataset, when the input parameters are not a str of csv path, but a pandas.Series, how to deel with it,this secion 
######test it

#print(seq_train_dataset)
'''
df=pd.DataFrame(seq_train_dataset)
print(df)

import os
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader    
class SeqDataset(torch.utils.data.Dataset):
    def __init__(self,seqPath):
        if type(seqPath)==str:
            self.df_file=pd.read_csv(seqPath,sep=',',header=None)
            self.sequence_list=self.df_file.iloc[:,0]
        else:
            self.df=pd.DataFrame(seqPath)
            self.sequence_list=self.df.iloc[:,0].tolist()
            print('self.sequence_list:',self.sequence_list)
    def __getitem__(self, idx):
        sequence=self.sequence_list[idx]
        
        return sequence
    
    def __len__(self):
        return len(self.sequence_list)

seq_train_dataset1=SeqDataset(seq_train_dataset)

seq_train_dataloader=DataLoader(seq_train_dataset1)
for seq in seq_train_dataloader:
    print(seq)
'''

"\ndf=pd.DataFrame(seq_train_dataset)\nprint(df)\n\nimport os\nimport torch\nimport pandas as pd\nfrom torch.utils.data import Dataset\nfrom torch.utils.data import DataLoader    \nclass SeqDataset(torch.utils.data.Dataset):\n    def __init__(self,seqPath):\n        if type(seqPath)==str:\n            self.df_file=pd.read_csv(seqPath,sep=',',header=None)\n            self.sequence_list=self.df_file.iloc[:,0]\n        else:\n            self.df=pd.DataFrame(seqPath)\n            self.sequence_list=self.df.iloc[:,0].tolist()\n            print('self.sequence_list:',self.sequence_list)\n    def __getitem__(self, idx):\n        sequence=self.sequence_list[idx]\n        \n        return sequence\n    \n    def __len__(self):\n        return len(self.sequence_list)\n\nseq_train_dataset1=SeqDataset(seq_train_dataset)\n\nseq_train_dataloader=DataLoader(seq_train_dataset1)\nfor seq in seq_train_dataloader:\n    print(seq)\n"

In [14]:
def train(args, epoch, model, device, loader, optimizer):
    model.train()
    #epoch_iter = tqdm(loader, desc="Iteration")
    for step, (A,B,C) in enumerate(loader):
        
        seq_data_list=[]
        seq=A
        lenth=len(seq)
        for m , s in enumerate(seq):
            seq_data_list.append((str(m),s))
        B=B.to(device)
        D,E=C
        D=D.to(device)
        E=E.to(device)
        C=(D,E)
        #print('D!!!!!!!!!!!!!!!!:',D)
        #print('E#####################:',E)
        
        pred=model(seq_data_list,B,C)#model is error
        y_true = B.y.view(pred.shape).to(torch.float64)
        loss = criterion(pred, y_true)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        #epoch_iter.set_description(f"Epoch: {epoch} tloss: {loss:.4f}")
        '''
        if (epoch + 1) % 10 == 0:
            print(f'training Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        '''

In [15]:
def eval(args, model, device, loader):
    model.eval()
    y_true = []
    y_scores = []
    
    #for step, batch in enumerate(tqdm(loader, desc="Iteration")):
    with torch.no_grad():
        for step, (A,B,C) in enumerate(loader):
            seq_data_list=[]
            seq=A
            lenth=len(seq)
            for m , s in enumerate(seq):
                seq_data_list.append((str(m),s))
            B=B.to(device)
            D,E=C
            D=D.to(device)
            E=E.to(device)##################
            C=(D,E)
        
        
            pred=model(seq_data_list,B,C)#model is error
            y_true=B.y.view(pred.shape).to(torch.float64)
            val_loss = criterion(pred, y_true)
            #print(f'Validation Loss: {val_loss.item():.4f}')    
                             

    #y_true = torch.cat(y_true, dim = 0).cpu().numpy()
    #y_scores = torch.cat(y_scores, dim = 0).detach().cpu().numpy()
    '''
    roc_list = []
    for i in range(y_true.shape[1]):
        #AUC is only defined when there is at least one positive data.
        if np.sum(y_true[:,i] == 1) > 0 and np.sum(y_true[:,i] == -1) > 0:
            is_valid = y_true[:,i]**2 > 0
            roc_list.append(roc_auc_score((y_true[is_valid,i] + 1)/2, y_scores[is_valid,i]))
    if len(roc_list)==0:#########################
        return 0
    if len(roc_list) < y_true.shape[1]:
        print("Some target is missing!")
        miss_ratio=(1 - float(len(roc_list))/y_true.shape[1])
        print("Missing ratio: %f" %(1 - float(len(roc_list))/y_true.shape[1]))
    '''
    
    return val_loss.item()

In [39]:
'''
import torch, gc
#criterion = nn.BCEWithLogitsLoss(reduction = "none")
criterion=nn.SmoothL1Loss()
gc.collect()
torch.cuda.empty_cache()

train_acc_list = []
val_acc_list = []
test_acc_list = []

if not args.filename == "":
    fname = 'runs/seq_mol_finetune_cls_runseed' + str(args.runseed) + '/' + args.filename
    #delete the directory if there exists one
    if os.path.exists(fname):
        shutil.rmtree(fname)
        print("removed the existing file.")
    writer = SummaryWriter(fname)

for epoch in range(1, args.epochs+1):
    print("====epoch " + str(epoch))
        
    train(args, epoch, model, device, seq_mol_smile_train_multi_loader, optimizer)

    print("====Evaluation")
    if args.eval_train:
        train_acc = eval(args, model, device, seq_mol_smile_train_multi_loader)
    else:
        print("omit the training accuracy computation")
        train_acc = 0
    val_acc = eval(args, model, device, seq_mol_smile_valid_multi_loader)
    test_acc = eval(args, model, device, seq_mol_smile_test_multi_loader)

    print("train: %f val: %f test: %f" %(train_acc, val_acc, test_acc))
    val_acc_list.append(val_acc)
    test_acc_list.append(test_acc)
    train_acc_list.append(train_acc)

    if not args.filename == "":
        writer.add_scalar('data/train auc', train_acc, epoch)
        writer.add_scalar('data/val auc', val_acc, epoch)
        writer.add_scalar('data/test auc', test_acc, epoch)

print('Best epoch:', val_acc_list.index(max(val_acc_list)))
print('Best auc: ', test_acc_list[val_acc_list.index(max(val_acc_list))])

exp_path = os.getcwd() + '/{}_results/{}/'.format(args.input_model_file, args.dataset)
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

df = pd.DataFrame({'train':train_acc_list,'valid':val_acc_list,'test':test_acc_list})
df.to_csv(exp_path + 'seed{}.csv'.format(args.runseed))

logs = 'Dataset:{}, Seed:{}, Best Epoch:{}, Best Acc:{:.5f}'.format(args.dataset, args.runseed, val_acc_list.index(max(val_acc_list)), test_acc_list[val_acc_list.index(max(val_acc_list))])
with open(exp_path + '{}_log.csv'.format(args.dataset),'a+') as f:
    f.write('\n')
    f.write(logs)

if not args.filename == "":
    writer.close()
'''

'\nimport torch, gc\n#criterion = nn.BCEWithLogitsLoss(reduction = "none")\ncriterion=nn.SmoothL1Loss()\ngc.collect()\ntorch.cuda.empty_cache()\n\ntrain_acc_list = []\nval_acc_list = []\ntest_acc_list = []\n\nif not args.filename == "":\n    fname = \'runs/seq_mol_finetune_cls_runseed\' + str(args.runseed) + \'/\' + args.filename\n    #delete the directory if there exists one\n    if os.path.exists(fname):\n        shutil.rmtree(fname)\n        print("removed the existing file.")\n    writer = SummaryWriter(fname)\n\nfor epoch in range(1, args.epochs+1):\n    print("====epoch " + str(epoch))\n        \n    train(args, epoch, model, device, seq_mol_smile_train_multi_loader, optimizer)\n\n    print("====Evaluation")\n    if args.eval_train:\n        train_acc = eval(args, model, device, seq_mol_smile_train_multi_loader)\n    else:\n        print("omit the training accuracy computation")\n        train_acc = 0\n    val_acc = eval(args, model, device, seq_mol_smile_valid_multi_loader)\n   

In [ ]:
import torch, gc
#criterion = nn.BCEWithLogitsLoss(reduction = "none")
criterion=nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
gc.collect()
torch.cuda.empty_cache()

train_acc_list = []
val_acc_list = []
test_acc_list = []

if not args.filename == "":
    fname = 'runs/seq_mol_finetune_cls_runseed' + str(args.runseed) + '/' + args.filename
    #delete the directory if there exists one
    if os.path.exists(fname):
        shutil.rmtree(fname)
        print("removed the existing file.")
    writer = SummaryWriter(fname)

for epoch in range(1, args.epochs+1):
    print("====epoch " + str(epoch))
        
    train(args, epoch, model, device, seq_mol_smile_train_multi_loader, optimizer)

    print("====Evaluation")
    if args.eval_train:
        train_loss = eval(args, model, device, seq_mol_smile_train_multi_loader)
    else:
        print("omit the training accuracy computation")
        train_loss = 0
    val_loss = eval(args, model, device, seq_mol_smile_valid_multi_loader)
    test_loss = eval(args, model, device, seq_mol_smile_test_multi_loader)

    print("train: %f val: %f test: %f" %(train_loss, val_loss, test_loss))
    

====epoch 1


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full 

====Evaluation


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full 

train: 0.009085 val: 2.180295 test: 3.602106
====epoch 2


...

/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full 

train: 0.014005 val: 0.000521 test: 2.997903
====epoch 34


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full 

====Evaluation


/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full dataset. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/home/wuyou/anaconda3/envs/wuyou_pytorch/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The given 'InMemoryDataset' only references a subset of examples of the full dataset, but 'data' will contain information of the full 

In [ ]:
#问题： 交叉注意力是加还是乘

#torch.tensor
x=torch.tensor([1,2])
x=torch.tensor([[1,2],[3,4]])
print(x)

In [ ]:
#test from torch_geometric.data import Data
from torch_geometric.data import Data
x = [
    [-1],
    [0],
    [1]
]
edge = [
  [0 , 1 , 0],
  [1 , 0 , 1],
  [0 , 1 , 0]
]
data = Data(x , edge)
print(data.x)






In [ ]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)
print(data.x)

In [14]:
####test device.type

x=torch.tensor([1,3,5])
x.to(device)
x=x.to(device)#############################################
print(x.device.type)

cpu


In [12]:
import torch

# 创建示例张量
x1 = torch.randn(1, 3)
x2 = torch.randn(1, 3)
print(x1.shape)
# 计算元素积
elementwise_product = x1 * x2

print(elementwise_product)

torch.Size([1, 3])
tensor([[-1.8426,  0.2394,  0.5487]])
